# Technical Factor - Triple Exponential Average (TRIX)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        m_ta_ema(close, 12) AS EMA1,
        m_ta_ema(EMA1, 12) AS EMA2,
        m_ta_ema(EMA2, 12) AS EMA3,
        (EMA3 - m_lag(EMA3, 1)) / m_lag(EMA3, 1) AS TRIX,
        m_avg(TRIX, 12) AS MATRIX,
        TRIX - MATRIX AS TRIX_HIST,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(TRIX > 0 AND m_lag(TRIX, 1) <= 0, 1, 0) AS TRBY1,
        IF(TRIX < 0 AND m_lag(TRIX, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(TRIX > MATRIX AND m_lag(TRIX, 1) <= m_lag(MATRIX, 1), 1, 0) AS MTBY1,
        IF(TRIX < MATRIX AND m_lag(TRIX, 1) >= m_lag(MATRIX, 1), 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(TRIX < 0 AND TRIX_HIST > m_lag(TRIX_HIST, 1) AND TRIX_HIST < 0, 1, 0) AS RVBY1,
        IF(TRIX > 0 AND TRIX_HIST < m_lag(TRIX_HIST, 1) AND TRIX_HIST > 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(TRIX, 1) <= 0 AND TRIX > 0 AND (TRIX - m_lag(TRIX, 1)) > 0, 1, 0) AS BKBY1,
        IF(m_lag(TRIX, 1) >= 0 AND TRIX < 0 AND (TRIX - m_lag(TRIX, 1)) < 0, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        EMA1,
        EMA2,
        EMA3,
        TRIX,
        MATRIX,
        TRIX_HIST,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,EMA1,EMA2,EMA3,TRIX,MATRIX,TRIX_HIST,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-03-14,600519.SH,12455.305695,12162.830404,12008.731618,0.002339,0.001401,0.000938,0,0,0,0,0,0,0,0
1,2025-03-17,600519.SH,12591.698833,12228.810162,12042.589855,0.002819,0.001549,0.001270,0,0,0,0,0,0,0,0
2,2025-03-18,600519.SH,12689.738823,12299.722264,12082.148687,0.003285,0.001732,0.001553,0,0,0,0,0,0,0,0
3,2025-03-19,600519.SH,12787.370909,12374.745133,12127.163525,0.003726,0.001947,0.001779,0,0,0,0,0,0,0,0
4,2025-03-20,600519.SH,12830.243163,12444.821753,12176.034022,0.004030,0.002187,0.001843,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2025-12-25,600519.SH,11875.477542,11878.009421,11916.197707,-0.000582,-0.000939,0.000357,0,0,0,0,0,0,0,0
194,2025-12-26,600519.SH,11886.195574,11879.268829,11910.516341,-0.000477,-0.000909,0.000432,0,0,0,0,0,0,0,0
195,2025-12-29,600519.SH,11879.501291,11879.304592,11905.714533,-0.000403,-0.000859,0.000456,0,0,0,0,0,0,0,0
196,2025-12-30,600519.SH,11857.878581,11876.008283,11901.144341,-0.000384,-0.000800,0.000416,0,0,0,0,0,0,0,0
